<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.12. Wave optics: Diffraction

Building upon our understanding of interference and polarization, which highlighted the wave nature of light, we now turn to another key wave phenomenon: **diffraction**. 
While interference involves the superposition of waves from multiple sources, diffraction describes the bending of waves as they pass through an aperture or around an obstacle. 
This bending is a direct consequence of the wave nature of light and becomes particularly noticeable when the size of the aperture or obstacle is comparable to the wavelength of the light. 


## 2.12.X Revisiting the double-slit experiment: Diffraction

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...
